In [1]:
import numpy as np
import pandas as pd

## The previous function doesn't check to ensure that there are no missing
##     differnt counts (ie, if let's say that number 4, didn't come through)
## Make up some sample data, with a missing message
room_names = ["Room1", "Room1", "Room1", "Room1", "Room1",
              "Room1", "Room1", "Room1", "Room1", "Room1",
              "Room1", "Room1", "Room1", "Room1", "Room1",
              "Room2", "Room2", "Room2", "Room2", "Room2",
              "Room2", "Room2", "Room2", "Room2", "Room2",
              "Room2", "Room2", "Room2",          "Room2",
              "Room3", "Room3", "Room3", "Room3", "Room3",
              "Room3", "Room3", "Room3", "Room3", "Room3",
                       "Room4", "Room4", "Room4", ]
user_names = ["User1", "User1", "User1", "User1", "User1",
              "User1", "User1", "User1", "User1", "User1",
              "User2", "User2", "User2", "User2", "User2",
              "User3", "User3", "User3", "User3", "User3",
              "User2", "User2", "User2", "User2", "User2",
              "User3", "User3", "User3",          "User3",
              "User4", "User4", "User4", "User4", "User4",
              "User4", "User4", "User4", "User4", "User4",
                       "User5", "User5", "User5", ]
messages   = ["1. Echo1",  "2. SMACK1",  "3. TEST1", "4. N/A1", "5. Time to Completion1: DEC 11 2024 20:05:00Z",
              "1. Echo2",  "2. SMACK2",  "3. TEST2", "4. N/A2", "5. Time to Completion2: DEC 11 2024 20:15:00Z",
              "1. Tango3", "2. SMACK3",  "3. TEST3", "4. N/A3", "5. Time to Completion3: DEC 11 2024 20:25:00Z",
              "1. Bravo4", "2. ESCORT4", "3. TEST4", "4. N/A4", "5. Time to Completion4: DEC 11 2024 20:06:00Z",
              "1. Echo5",  "2. SMACK5",  "3. TEST5", "4. N/A5", "5. Time to Completion5: DEC 11 2024 20:16:00Z",
              "1. Echo7",  "2. SMACK7",  "3. TEST7",            "5. Time to Completion7: DEC 11 2024 20:26:00Z",
              "1. Tango6", "2. SMACK6",  "3. TEST6", "4. N/A6", "5. Time to Completion6: DEC 11 2024 20:17:00Z",
              "1. Tango8", "2. SMACK8",  "3. TEST8", "4. N/A8", "5. Time to Completion8: DEC 11 2024 20:17:00Z",
                           "2. SMACK9",  "3. TEST9", "4. N/A9", ]
timestamps = ["2024-12-11 20:00:00.000Z", "2024-12-11 20:00:00.000Z", "2024-12-11 20:00:00.000Z", "2024-12-11 20:00:00.001Z", "2024-12-11 20:00:00.001Z",
              "2024-12-11 20:00:01.000Z", "2024-12-11 20:00:01.000Z", "2024-12-11 20:00:01.002Z", "2024-12-11 20:00:01.002Z", "2024-12-11 20:00:01.002Z",
              "2024-12-11 20:15:01.000Z", "2024-12-11 20:15:01.000Z", "2024-12-11 20:15:01.002Z", "2024-12-11 20:15:01.002Z", "2024-12-11 20:15:01.002Z",
              "2024-12-11 20:00:00.000Z", "2024-12-11 20:00:00.000Z", "2024-12-11 20:00:00.002Z", "2024-12-11 20:00:00.002Z", "2024-12-11 20:00:00.002Z",
              "2024-12-11 20:00:00.010Z", "2024-12-11 20:00:00.010Z", "2024-12-11 20:00:00.011Z", "2024-12-11 20:00:00.011Z", "2024-12-11 20:00:00.011Z",
              "2024-12-11 20:00:01.010Z", "2024-12-11 20:00:01.010Z", "2024-12-11 20:00:01.011Z",                             "2024-12-11 20:00:01.011Z",
              "2024-12-11 20:00:00.010Z", "2024-12-11 20:00:00.010Z", "2024-12-11 20:00:00.011Z", "2024-12-11 20:00:00.011Z", "2024-12-11 20:00:00.011Z",
              "2024-12-11 20:15:00.020Z", "2024-12-11 20:15:00.020Z", "2024-12-11 20:15:00.020Z", "2024-12-11 20:15:00.020Z", "2024-12-11 20:15:00.020Z",
                                          "2024-12-11 20:40:00.020Z", "2024-12-11 20:40:00.020Z", "2024-12-11 20:40:00.020Z", ]

print(f"len(room_names): {len(room_names)}")
print(f"len(user_names): {len(user_names)}")
print(f"len(messages): {len(messages)}")
print(f"len(timestamps): {len(timestamps)}")

len(room_names): 42
len(user_names): 42
len(messages): 42
len(timestamps): 42


In [2]:
## #############################################################################
## Convert our messages, into a ten line format.  Prefixes and newcolumns are defaulted,
##     but you can adjust them to whatever format you want.
##     Best matches will only return those matches that are smallest
## #############################################################################
def convert_messages_to_ten_line(msg_df,
            prefixes=["1. ", "2. ", "3. ", "4. ", "5. ",
                      "6. ", "7. ", "8. ", "9. ", "10. "],
            new_columns=["msg_1", "msg_2", "msg_3", "msg_4", "msg_5",
                         "msg_6", "msg_7", "msg_8", "msg_9", "msg_10"],
            best_matches=False,
            msg_col="Message", timestmp_col="Timestamp",
            groupby_cnt_col="msg_cnt",
            match_group_col="match_group", time_diff_col="time_diff",
            groupby=["Room Name", "User Name"]):

    ## First go ahead and create the new columns and strip off our prefixes
    new_df = msg_df.copy()
    ## Loop through the prefixes and create the new columns
    for i, prefix in enumerate(prefixes):
        new_df[new_columns[i]] = new_df[new_df[msg_col].str.startswith(prefix)][msg_col].str.slice(len(prefix))

    ## Drop the message column, since we don't need it anymore
    new_df = new_df.drop(columns=[msg_col])
    #print(f"==================== new df {len(new_df)} records ====================")
    #display(new_df)

    ## Build our groupby (that includes the timestamp column)
    full_groupby = groupby.copy()
    full_groupby.append(timestmp_col)

    ## Add a column for our groupby count
    new_df[groupby_cnt_col] = 0

    ## GroupBy, first build a dic for our group by
    agg_dict = {}
    for col in new_df.columns:
        if not (col in full_groupby):
          agg_dict[col] = "first"
    agg_dict[groupby_cnt_col] = "count"
    new_df = new_df.groupby(full_groupby).agg(agg_dict).reset_index()
    #print("\n")
    #print(f"==================== groupedby df {len(new_df)} records ====================")
    #display(new_df)

    ## Add the new columns we need for building / calculating matches for our dataframes
    new_df[time_diff_col] = [{} for _ in range(len(new_df))]
    new_df[match_group_col] = 0

    ## If we have rows that equal the number of message we were looking for, then set them aside
    complete_df = new_df[new_df[groupby_cnt_col] == len(prefixes)]
    #display(f"==================== complete df {len(complete_df)} records ====================")
    #display(complete_df)

    new_df = new_df[new_df[groupby_cnt_col] < len(prefixes)]
    print("\n")
    print(f"==================== partial df {len(new_df)} records ====================")
    display(new_df)

    ## ======== Calculate the time difference between matching rows ========
    ## Loop through all of the rows and calculate the different time differences between the
    ##     rows that were grouped together
    for index, row in new_df.iterrows():
        #print(f"index: {index}")
        time_differnces = {}
        for index2, row2 in new_df[index + 1:].iterrows():
            #print(f"index2: {index2}")
            if index != index2:
                correct_group = True
                for col in groupby:
                    #print(f"{index}.{row[col]} != {index2}.{row2[col]} ({row[col] != row2[col]})")
                    if row2[col] != row[col]:
                        correct_group = False
                        break

                if correct_group:
                    time_differnces[index2] = (row2[timestmp_col] - row[timestmp_col]).total_seconds() * 1000
                    new_df.at[index, match_group_col] += 1
                    new_df.at[index, time_diff_col][index2] = abs((row2[timestmp_col] - row[timestmp_col]).total_seconds() * 1000)
                    new_df.at[index2, match_group_col] += 1
                    new_df.at[index2, time_diff_col][index] = abs((row2[timestmp_col] - row[timestmp_col]).total_seconds() * 1000)

    ## Take any records that had no matches and move them to the complete dataframe
    tmp_df = new_df[new_df[match_group_col] == 0]
    complete_df = pd.concat([complete_df, tmp_df])

    ## Drop the records where there was no match found
    new_df = new_df[new_df[match_group_col] > 0]

    print("\n")
    print(f"==================== unmatched/complete df {len(complete_df)} records, with timedifferences ====================")
    display(complete_df)

    print("\n")
    print(f"==================== partial df {len(new_df)} records, with timedifferences ====================")
    display(new_df)


    ## ======== Loop through all of our unmatched records and see about ========
    ## Setup our merged dataframe
    merged = {}
    for column in new_df.columns:
        merged[column] = []
    rows_inspected = []
    for index, row in new_df.iterrows():
        ## If we've already used this row, then move along
        if index not in rows_inspected:
            ## Matching Row
            time_differnces = row[time_diff_col]
            min_time_diff = min(time_differnces.values())
            key = [key for key, value in time_differnces.items() if value == min_time_diff][0]
            match_row = new_df.loc[key]

            #rows_inspected.append(index)

    return None


df_org = pd.DataFrame.from_dict({"Room Name": room_names, "User Name": user_names, "Message": messages, "Timestamp": timestamps}, )
df_org['Timestamp'] = pd.to_datetime(df_org['Timestamp'])

prefixes = ["1. ", "2. ", "3. ", "4. ", "5. "]
new_columns = ["msg_1", "msg_2", "msg_3", "msg_4", "msg_5"]

convert_messages_to_ten_line(df_org, prefixes=prefixes, new_columns=new_columns)



==================== partial df 15 records ====================


,Room Name,User Name,Timestamp,msg_1,msg_2,msg_3,msg_4,msg_5,msg_cnt,time_diff,match_group
0,Room1,User1,2024-12-11 20:00:00+00:00,Echo1,SMACK1,TEST1,None,None,3,{},0
1,Room1,User1,2024-12-11 20:00:00.001000+00:00,None,None,None,N/A1,Time to Completion1: DEC 11 2024 20:05:00Z,2,{},0
2,Room1,User1,2024-12-11 20:00:01+00:00,Echo2,SMACK2,None,None,None,2,{},0
3,Room1,User1,2024-12-11 20:00:01.002000+00:00,None,None,TEST2,N/A2,Time to Completion2: DEC 11 2024 20:15:00Z,3,{},0
4,Room1,User2,2024-12-11 20:15:01+00:00,Tango3,SMACK3,None,None,None,2,{},0
5,Room1,User2,2024-12-11 20:15:01.002000+00:00,None,None,TEST3,N/A3,Time to Completion3: DEC 11 2024 20:25:00Z,3,{},0
6,Room2,User2,2024-12-11 20:00:00.010000+00:00,Echo5,SMACK5,None,None,None,2,{},0
7,Room2,User2,2024-12-11 20:00:00.011000+00:00,None,None,TEST5,N/A5,Time to Completion5: DEC 11 2024 20:16:00Z,3,{},0
8,Room2,User3,2024-12-11 20:00:00+00:00,Bravo4,ESCORT4,None,None,None,2,{},0
9,Room2,User3,2024-12-11 20:00:00.002000+00:00,None,None,TEST4,N/A4,Time to Completion4: DEC 11 2024 20:06:00Z,3,{},0




==================== unmatched/complete df 2 records, with timedifferences ====================


,Room Name,User Name,Timestamp,msg_1,msg_2,msg_3,msg_4,msg_5,msg_cnt,time_diff,match_group
14,Room3,User4,2024-12-11 20:15:00.020000+00:00,Tango8,SMACK8,TEST8,N/A8,Time to Completion8: DEC 11 2024 20:17:00Z,5,{},0
15,Room4,User5,2024-12-11 20:40:00.020000+00:00,None,SMACK9,TEST9,N/A9,None,3,{},0




==================== partial df 14 records, with timedifferences ====================


,Room Name,User Name,Timestamp,msg_1,msg_2,msg_3,msg_4,msg_5,msg_cnt,time_diff,match_group
0,Room1,User1,2024-12-11 20:00:00+00:00,Echo1,SMACK1,TEST1,None,None,3,"{1: 1.0, 2: 1000.0, 3: 1002.0}",3
1,Room1,User1,2024-12-11 20:00:00.001000+00:00,None,None,None,N/A1,Time to Completion1: DEC 11 2024 20:05:00Z,2,"{0: 1.0, 2: 999.0, 3: 1000.9999999999999}",3
2,Room1,User1,2024-12-11 20:00:01+00:00,Echo2,SMACK2,None,None,None,2,"{0: 1000.0, 1: 999.0, 3: 2.0}",3
3,Room1,User1,2024-12-11 20:00:01.002000+00:00,None,None,TEST2,N/A2,Time to Completion2: DEC 11 2024 20:15:00Z,3,"{0: 1002.0, 1: 1000.9999999999999, 2: 2.0}",3
4,Room1,User2,2024-12-11 20:15:01+00:00,Tango3,SMACK3,None,None,None,2,{5: 2.0},1
5,Room1,User2,2024-12-11 20:15:01.002000+00:00,None,None,TEST3,N/A3,Time to Completion3: DEC 11 2024 20:25:00Z,3,{4: 2.0},1
6,Room2,User2,2024-12-11 20:00:00.010000+00:00,Echo5,SMACK5,None,None,None,2,{7: 1.0},1
7,Room2,User2,2024-12-11 20:00:00.011000+00:00,None,None,TEST5,N/A5,Time to Completion5: DEC 11 2024 20:16:00Z,3,{6: 1.0},1
8,Room2,User3,2024-12-11 20:00:00+00:00,Bravo4,ESCORT4,None,None,None,2,"{9: 2.0, 10: 1010.0, 11: 1010.9999999999999}",3
9,Room2,User3,2024-12-11 20:00:00.002000+00:00,None,None,TEST4,N/A4,Time to Completion4: DEC 11 2024 20:06:00Z,3,"{8: 2.0, 10: 1008.0, 11: 1008.9999999999999}",3
